In [34]:
import random
import numpy as np

random_seed = 42

Initialization

In [35]:
N = 5

In [65]:
def create_initiate_state(n: int, rd_seed: int | None = None):
    if rd_seed:
        random.seed(rd_seed)
    else:
        random.seed()
    all_possible_positions = [(i, j) for i in range(n) for j in range(n)]
    random.shuffle(all_possible_positions)
    return set(all_possible_positions[:n])


In [67]:
def get_matrix_view(positions: set, n: int):
    matrix = np.zeros((n, n), dtype=int)
    for i, j in positions:
        matrix[i][j] = 1
    return matrix

In [68]:
queens = create_initiate_state(N, random_seed)

In [69]:
get_matrix_view(queens, N)

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0]])

In [70]:
def get_state_neighbors(queen: tuple[int, int], n: int, obstacles: set | None = None):
    neighbors = set()
    q_row, q_col = queen
    horizontal_directions = [(0, 1), (0, -1)]
    vertical_directions = [(1, 0), (-1, 0)]
    diag_directions = [(1, 1), (-1, 1), (1, -1), (-1, -1)]
    directions = horizontal_directions + vertical_directions + diag_directions
    for direction in directions:
        d_row, d_col = direction
        c_row, c_col = q_row + d_row, q_col + d_col
        while 0 <= c_row < n and 0 <= c_col < n:
            if obstacles and (c_row, c_col) in obstacles:
                break
            neighbors.add((c_row, c_col))
            c_row, c_col = c_row + d_row, c_col + d_col
    return neighbors

In [71]:
def heuristic(queens: set):
    random.seed(random_seed)
    h = 0
    n = len(queens)
    remaining_queens: set = queens.copy()
    while len(remaining_queens) > 0:
        queen = remaining_queens.pop()
        neighbors = get_state_neighbors(queen, n)
        h += len(neighbors & remaining_queens)
    return h


In [72]:
heuristic(queens)

6

In [73]:
def get_all_neighbor_states(queens: set):
    n = len(queens)
    neighbors_states = list()
    for queen in queens:
        remaining_queens = queens - {queen}
        neighbors = get_state_neighbors(queen, n, remaining_queens)
        for neighbor in neighbors:
            neighbors_states.append(remaining_queens | {neighbor})
    return neighbors_states


In [74]:
def get_best_state(states: list):
    best_state = states.pop()
    best_h = heuristic(best_state)
    for state in states:
        h = heuristic(state)
        if h < best_h:
            best_h = h
            best_state = state
    return best_state, best_h

In [75]:
def hill_climbing(initial_state):
    current_state, current_h = initial_state, heuristic(initial_state)
    while True:
        all_neighbors = get_all_neighbor_states(current_state)
        best_neighbor_state, best_h = get_best_state(all_neighbors)
        if best_h >= current_h:
            return current_state, current_h
        current_state, current_h = best_neighbor_state, best_h


In [76]:
def n_queens(n: int, rd_seed: int | None = None):
    initial_state = create_initiate_state(n, rd_seed)
    best_state, best_h = hill_climbing(initial_state)
    print(best_h)
    print(get_matrix_view(best_state, n))

In [99]:
n_queens(10)

2
[[0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1]
 [0 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]]
